<a href="https://colab.research.google.com/github/edelmode/NavotasCity/blob/main/Copy_of_link_project_cchain_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/thinkingmachines/project-cchain/gh-pages/assets/full_logo.png" alt="drawing" width="400"/>

# Link Project CCHAIN dataset tables

> This code is under the [Apache 2.0](https://www.apache.org/licenses/LICENSE-2.0) license, which  allows developers to freely use, modify, and share software on an "as-is" basis, meaning without any warranties. When making changes, developers are required to document changes to the original code.

This notebook demonstrates a way to link Project CCHAIN tables to a single dataset.

This notebook includes a **Select Paremeters** section, offering options to generate weekly city-level dataset.

- Moreover, this notebook also provides the option to download the daily
barangay-level dataset used to aggregate weekly city-level dataset.

Please take note of the following:

1. Run **one code block at a time, in order,** to successfully generate the linked dataset output.

2. Some code blocks require user input via widgets. **Always run the code block containing the widget** before inputting your values.

> **_NOTE:_**  Google Colab is a free tool with limited RAM, which may crash if the requested dataset is too large. Please use it conservatively and request only the necessary data.


The tables have various spatial (barangay and city) and temporal (static, yearly, monthly) resolutions. For more details on each table, the full documentation for the Project CCHAIN dataset can be found [here](https://thinkingmachines.github.io/project-cchain/open_data.html).

To show its functionality, instructions are provided below to produce a **city-weekly dengue outcomes analysis linked dataset for Zamboanga City**.

# 1. Setting up

Before running the cells below, please do the following:
1. Create a folder at your preferred location in your Google Drive
2. Enter the folder and create the following folders inside
  *   `data/`
  *   `code/`

2. Download all files from [here](https://drive.google.com/drive/u/1/folders/1NF_e3YM0dVBSxlVBWRxGQXwkl59sp5Jf). Upload all .csv files to `data/`

3. Download this notebook file and upload to `code/`


In [ ]:
# @title 1.1. Load packages
# @markdown DO NOT SKIP RUNNING THIS BLOCK. This section loads python libraries and packages

# @markdown A window should appear requesting access to your Google Drive. Please provide your login credentials and select `Continue` to run this block.
# general
import os
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os
from functools import reduce
import geopandas as gpd
import random
import warnings
import re
import sys
import warnings

# geospatial
import geopandas as gpd

# visualization
#import matplotlib.pyplot as plt
#import seaborn as sns

# widgets
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Javascript

# google work environment
from google.colab import auth
from google.colab import drive
from google.colab import files

# mount google drive
drive.mount('/content/drive')

# supress all warnings
warnings.filterwarnings("ignore")



Mounted at /content/drive


In [ ]:
# @title 1.2. Select input folder
# @markdown DO NOT SKIP RUNNING THIS BLOCK. This section sets the data directory location.
print ("Sample input folder: /content/drive/My Drive/Colab Notebooks/data")

####### INPUT FOLDER DIRECTORY WIDGET ########
input_directory  = widgets.Text(
    placeholder='Type input folder here and press ENTER',
    description='Input folder:',
    disabled=False,
    layout={'width': '95%'}
)
def handle_input_directory_folder_enter(sender):
  assert sender, "Please enter a non-empty string."
  print(f'Checking directory: {sender.value.strip()}')
  assert os.path.exists(sender.value.strip()), f"ERROR: Directory does not exist.\nPlease enter a valid directory before proceeding with next cell."
  input_folder = sender.value.strip()
  print("Directory exists! Please proceed with the next code cell.")
input_directory.on_submit(handle_input_directory_folder_enter)
display(input_directory)


Sample input folder: /content/drive/My Drive/Colab Notebooks/data


Text(value='', description='Input folder:', layout=Layout(width='95%'), placeholder='Type input folder here an…

Checking directory: /content/drive/My Drive/Colab Notebooks/data
Directory exists! Please proceed with the next code cell.


In [ ]:
# @title 1.3. Load files
# @markdown DO NOT SKIP RUNNING THIS BLOCK. This section checks whether directory has complete files and reads them to memory.
input_folder = input_directory.value
assert len(input_folder)>0, "No input directory provided. Please run previous cell."

fn_list = [fn for fn in os.listdir(input_folder) if fn.endswith('.csv')]
assert len(fn_list) > 0, f"ERROR: No CSV files found in the specified folder.\n Check if you have placed the tables in the data/ folder."

print('Loading tables...')
# create progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=len(fn_list),
    bar_style='info',
    style={'bar_color': 'blue'},
    orientation='horizontal'
)
display(progress_bar)

# create list to store dataframes
dataframes = []
dfs_to_merge = []

# loop through files and convert to dataframe
for i, file_name in enumerate(fn_list):
    if file_name in ['tm_open_buildings.csv',
                    'climate_timestep_check.csv',
                    'disease_lgu_disaggregated_totals.csv',
                     'disease.csv',
                     'disease_fhsis_totals.csv']:
        continue

    progress_bar.value = i + 1  # Update progress bar
    file_path = os.path.join(input_folder, file_name)
    df_name = f"{file_name.split('.')[0].lower()}"
    df = pd.read_csv(file_path)
    df = df.drop(['ADM2_EN', 'ADM2_PCODE', 'Unnamed: 0'], axis=1, errors='ignore')
    dataframes.append((df_name, df))
    globals()[df_name] = df
    dfs_to_merge.append(df)

# convert all column names to lowercase
for df_name, DataFrame in dataframes:
    df.rename(columns={col: col.lower() for col in df.columns}, inplace=True)

# print total number of dataframes
print(f"Total number of DataFrames: {len(dataframes)}")

# Assert that exactly 24 dataframes are read
try:
    assert len(dataframes) == 24, f"Expected exactly 24 relevant dataframes, but only {len(dataframes)} were read."
    print("All 24 tables were complete and read successfully")
except AssertionError as e:
    print(e)
    print("Please check folder data/ if it contains all expected files.")


Loading tables...


IntProgress(value=0, bar_style='info', max=29, style=ProgressStyle(bar_color='blue'))

Total number of DataFrames: 24
All 24 tables were complete and read successfully


In [ ]:
# @title 1.4. Create connector table
# @markdown DO NOT SKIP RUNNING THIS BLOCK.

# @markdown The connector table combines the location and calendar tables, forming the base columns of the dataset.
print("Creating the connector table...")

# barangay table
brgy = location[['adm3_en','adm3_pcode','adm4_en','adm4_pcode', 'brgy_total_area']]
calendar = calendar[['date']]
base_brgy = pd.merge(brgy.assign(key=1), calendar.assign(key=1), on='key').drop('key', axis=1)

# city table
brgy = location[['adm3_en','adm3_pcode']]
calendar = calendar[['date']]
base_city = pd.merge(brgy.assign(key=1), calendar.assign(key=1), on='key').drop('key', axis=1)
base_city = base_city.drop_duplicates(subset=['adm3_en', 'date'], keep='first')

# differentiate barangay and city-level tables
count_with_adm4_pcode = 0
dfs_with_adm4_pcode = []
count_without_adm4_pcode = 0
dfs_without_adm4_pcode = []

for df_name, df in dataframes:
    if 'adm4_pcode' in df.columns:
        count_with_adm4_pcode += 1
        dfs_with_adm4_pcode.append((df_name, df))
    else:
        count_without_adm4_pcode += 1
        dfs_without_adm4_pcode.append((df_name, df))

dfs_with_adm4_pcode.insert(0, ('base_brgy', base_brgy))
dfs_without_adm4_pcode.insert(0, ('base_city', base_city))
print('Connector table successfully generated')

Creating the connector table...
Connector table successfully generated


# 2. Select dataset parameters

Here are some tips to decide on what parameters to use to effectively make use of the Project CCHAIN dataset.

1. **Focus on one location**. Starting with a detailed analysis of one location allows for a better understanding of the local dynamics, which may differ across locations.

2. **Do not use all variables at once- do a literature review first to identify possible key variables**. More often than not, using all variables is not necessary and may even yield subpar results. Moreover, it is not resource-efficient, and may cause cloud-run code like this notebook to crash.

3. **Check data availability on your focus location** and make sure they fit the requirements of your study. The details and schema for each dataset are provided in the dataset full documentation [here](https://dbdocs.io/lacuna-fund-project-team/project-cchain).




## **SCENARIO: Weekly city-level dengue outcomes analysis linked dataset for Zamboanga City**

The following are the parameters required by the scenario


| Parameter                            | Details                                                                                   |
|--------------------------------------|-------------------------------------------------------------------------------------------|
| Aggregation                          | City                                                                         |
| Start and end date                   | 01 Jan 2014 - 31 Dec 2022                                                                 |
| City                                 | Zamboanga City                                                                            |
| Tables and columns                   | `climate_atmosphere`: heat_index, pr, rh, solar_rad, tave, tmax, tmin, uv_rad, wind_speed |
|                                      | `disease_weekly_totals`: case_total                                                       |
|                                      | `nighttime_lights`: avg_rad_mean                                                          |
|                                      | `osm_poi_health`: (*xx refers to `_count` and `_nearest`*) clinic_xx, dentist_xx, doctors_xx, hospital_xx, optician_xx, pharmacy_xx  |
|                                      | `osm_poi_total`: poi_count                                                                |
|                                      | `worldpop_population`: pop_count_mean, pop_count_total                                    |
| Disease                              | Dengue                                                                                    |




**Instructions on how to select parameters**

1. Run code cell below

2. First, choose and select **select aggregation setting** among options:

* `Barangay (no health data)` - if you only need the environment and socioeconomic data at baranggay level
* `City` - if you need city aggregated health, environment and socioeconomic data

3. The remaining multi-select markdowns should now be activated. Use **CTRL + click** to select multiple tables and columns. You may also drag along the list while pressing CTRL to select multiple adjacent parameters.

4. The **available list of tables, columns, and diseases update automatically** as options are selected.


<sub> *Note 1:* Disease data from FHSIS (`disease_monthly_totals`) and LGU disaggregated sources (`disease_lgu_disaggregated_totals`) are also available [here](https://drive.google.com/drive/u/0/folders/1VaUIEkjZ7SJCOoy4_nXE80Mx8S5ksVFZ) but is not included in this notebook.

<sub> *Note 2:* This notebook excludes the `geometry` column, which consumes a lot of the limited computing power of this tool. You may choose to process geographical data separately on your local machine by taking a look at the `brgy_geography` table.


We advise that tables be downloaded ONLY at the required dates, cities, and parameters to limit processing overhead. The details and schema for each dataset are provided in the dataset full documentation [here](https://dbdocs.io/lacuna-fund-project-team/project-cchain).



In [ ]:
# @title 2.1. Select parameters and output folder

######## DATE WIDGET ########

start_date = datetime(2003, 1, 1)
end_date = datetime(2022, 12, 31)

start_date_picker = widgets.DatePicker(
    description='Start Date:',
    value=start_date,
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='End Date:',
    value=end_date,
    disabled=False
)


######## CITY OR BARANGAY DROPDOWN WIDGET ########

city_or_barangay_radio = widgets.Dropdown(
    options=['Barangay (no health data)', 'City'],
    value=None,
    description='Choose one:',
    disabled=False
)

######## CITY WIDGET ########

city_options = sorted(base_city['adm3_en'].unique())
selected_cities = widgets.Select(
    options=city_options,
    description='Choose City:',
    disabled=False
)

######## TABLE WIDGET ########

options = []
selected_tables = widgets.SelectMultiple(
    options=options,
    value=(),
    description='Choose table:',
    disabled=False
)

def update_table_options(change):
    global options

    if change.new == 'City':
        options = [df_name for df_name, _ in dfs_with_adm4_pcode + dfs_without_adm4_pcode
                   if df_name not in ['base_brgy', 'base_city', 'calendar', 'disease_monthly_totals', 'location']]
    elif change.new == 'Barangay (no health data)':
        options = [df_name for df_name, _ in dfs_with_adm4_pcode
                   if df_name not in ['base_brgy', 'location', 'mapbox_health_facility_brgy_isochrones']]

    selected_tables.options = sorted(options)
    selected_tables.value = ()

city_or_barangay_radio.observe(update_table_options, names='value')

######## COLUMNS WIDGET ########

dfs_with_adm4_pcode.sort(key=lambda x: x[0])
dfs_without_adm4_pcode.sort(key=lambda x: x[0])
for _, df in dfs_with_adm4_pcode + dfs_without_adm4_pcode:
    df.sort_index(axis=1, inplace=True)

dataframes_dict = {**{df_name: df for df_name, df in dfs_with_adm4_pcode},
                   **{df_name: df for df_name, df in dfs_without_adm4_pcode}}


selected_columns = widgets.SelectMultiple(
    options=[],
    value=[],
    description='Select Columns:',
    disabled=True
)

selected_columns_value = ['adm3_en', 'adm4_pcode', 'adm4_pcode', 'brgy_total_area', 'date', 'source_name', 'disease_standard_code', 'disease_standard_name']

def update_columns_options(change):
    global selected_columns_value

    selected_table_names = change.new
    if selected_table_names:
        columns = []
        for table_name in selected_table_names:
            df = dataframes_dict[table_name]
            columns.extend(list(df.columns))

        columns_to_remove = ['adm1_en', 'adm1_pcode', 'adm2_en', 'adm2_pcode','adm3_pcode', 'adm4_en', 'adm4_pcode',
                             'brgy_total_area', 'date', 'freq', 'year', 'year_month', 'uuid', 'source_name',
                             'disease_standard_code', 'disease_standard_name', 'geometry']
        columns = [col for col in columns if col not in columns_to_remove]

        table_columns = {}

        for col in columns:
            table_name = next((name for name, df in dataframes_dict.items() if col in df.columns), None)
            if table_name not in table_columns:
                table_columns[table_name] = []
            table_columns[table_name].append(col)

        for table_name, cols in table_columns.items():
            table_columns[table_name] = sorted(cols)

        sorted_tables = sorted(table_columns.keys())

        sorted_columns = [col for table in sorted_tables for col in table_columns[table]]

        selected_columns.options = sorted_columns

        selected_columns.disabled = False

selected_tables.observe(update_columns_options, names='value')


######## DISEASES WIDGET ########

selected_diseases = widgets.SelectMultiple(
    options=[],  # Placeholder for disease options
    value=[],  # Placeholder for selected diseases
    description='Select Diseases:',
    disabled=True
)

output_directory = widgets.Text(
    placeholder='Type output folder here and press ENTER',
    description='Output folder:',
    disabled=False,
    layout={'width': '95%'}  # Adjust size as needed
)

def update_diseases_options(change):
    global disease_weekly_totals

    selected_column_names = change.new
    disease_options = []

    for column in selected_column_names:
        if column == 'case_total':
            filtered = disease_weekly_totals.dropna(subset=['case_total'])
            filtered = filtered[~filtered['source_name'].str.contains('FHSIS', regex=True)]
            disease_options.extend(sorted(filtered['disease_standard_name'].unique()))
        elif column == 'death_total':
            filtered = disease_weekly_totals.dropna(subset=['death_total'])
            filtered = filtered[~filtered['source_name'].str.contains('FHSIS', regex=True)]
            disease_options.extend(sorted(filtered['disease_standard_name'].unique()))

    selected_diseases.options = sorted(set(disease_options))
    selected_diseases.disabled = False
    # change output directory text status and message
    output_directory.disabled = selected_diseases.disabled
    output_directory.placeholder = 'Type output folder here and press ENTER'

selected_columns.observe(update_diseases_options, names='value')

####### OUTPUT FOLDER DIRECTORY WIDGET ########

output_directory = widgets.Text(
    placeholder='Please complete selections first!',
    description='Output folder:',
    disabled=True,
    layout={'width': '95%'}  # Adjust size as needed
)

def handle_output_directory_folder_enter(sender):
  assert sender, "Please enter a non-empty string."
  print(f'Checking directory: {sender.value.strip()}')
  assert os.path.exists(sender.value.strip()), f"ERROR: Directory does not exist.\nPlease enter a valid directory before proceeding with next cell."
  input_folder = sender.value.strip()
  print("Directory exists! Please proceed to the next cell.")

output_directory.on_submit(handle_output_directory_folder_enter)
####### LAYOUT #######

selected_tables.layout.width = '20%'
selected_tables.layout.height = '400px'
selected_columns.layout.width = '25%'
selected_columns.layout.height = '400px'
selected_cities.layout.width = '15%'
selected_cities.layout.height = '400px'
start_date_picker.layout.width = '15%'
start_date_picker.layout.height = 'auto'
end_date_picker.layout.width = '15%'
end_date_picker.layout.height = 'auto'
city_or_barangay_radio.layout.width = '20%'
city_or_barangay_radio.layout.height = 'auto'
selected_diseases.layout.width = '15%'
selected_diseases.layout.height = '400px'

display(
    widgets.VBox([
        widgets.HBox([start_date_picker, city_or_barangay_radio]),
        widgets.HBox([end_date_picker, selected_cities, selected_tables, selected_columns, selected_diseases]),
    ])
)

print ("\n")
print("Sample output folder: /content/gdrive/Shared drives/[EXT] Lacuna Fund - Project CCHAIN/processed")

display(
    widgets.VBox([
        output_directory
    ])
)



Sample output folder: /content/gdrive/Shared drives/[EXT] Lacuna Fund - Project CCHAIN/processed


NameError: name 'disease_weekly_totals' is not defined

NameError: name 'disease_weekly_totals' is not defined

NameError: name 'disease_weekly_totals' is not defined

NameError: name 'disease_weekly_totals' is not defined

...

NameError: name 'disease_weekly_totals' is not defined

NameError: name 'disease_weekly_totals' is not defined

NameError: name 'disease_weekly_totals' is not defined

NameError: name 'disease_weekly_totals' is not defined

Checking directory: /content/drive/My Drive/Colab Notebooks/data
Directory exists! Please proceed to the next cell.


In [ ]:
# @title 2.2. Extract parameters from selection

# extract general options
city_or_barangay = ''.join(city_or_barangay_radio.value)
output_folder = output_directory.value

# barangay- extract values from selections
tables_to_exclude = ["disease_weekly_totals", "mapbox_health_facility_city_isochrones"]
columns_from_mapbox_health_facility_city_isochrones = list(mapbox_health_facility_city_isochrones)


tables = [table for table in selected_tables.value if table not in tables_to_exclude]
columns = [column for column in selected_columns.value if column not in columns_from_mapbox_health_facility_city_isochrones or column not in columns_from_disease_weekly_totals]
cities = [selected_cities.value] if not isinstance(selected_cities.value, list) else selected_cities.value
start_date = start_date_picker.value.strftime('%Y-%m-%d')
end_date = end_date_picker.value.strftime('%Y-%m-%d')

# city- extract values from selections
tables_to_include = ["disease_weekly_totals", "mapbox_health_facility_city_isochrones"]
columns_from_mapbox_health_facility_city_isochrones = list(mapbox_health_facility_city_isochrones)


city_tables = [table for table in selected_tables.value if table in tables_to_include]
city_columns = [column for column in selected_columns.value if column in columns_from_mapbox_health_facility_city_isochrones]
# start_date = start_date_picker.value.strftime('%Y-%m-%d')
# end_date = end_date_picker.value.strftime('%Y-%m-%d')
diseases = list(selected_diseases.value)

print( "The following are the dataset specifications based on the chosen parameters:")
print(f" ● Aggregation: {city_or_barangay}")
print(f" ● Inclusive dates: {start_date} to {end_date}")
print(f" ● City: {cities}")
print(f" ● Disease columns: {diseases}")
for i in range(0, len(columns), 5):
  if i == 0:
    print(f" ● Factors columns: {columns[i:i+5]}")
  else:
    print(f"                    {columns[i:i+5]}")



The following are the dataset specifications based on the chosen parameters:
 ● Aggregation: City
 ● Inclusive dates: 2014-01-01 to 2022-12-31
 ● City: ['City of Navotas']
 ● Disease columns: []
 ● Factors columns: ['heat_index', 'pr', 'rh', 'solar_rad', 'tave']
                    ['tmax', 'tmin', 'uv_rad', 'wind_speed', 'case_total']
                    ['avg_rad_mean', 'clinic_nearest', 'dentist_nearest', 'doctors_nearest', 'hospital_nearest']
                    ['optician_nearest', 'pharmacy_nearest', 'poi_count', 'pop_count_mean', 'pop_count_total']


# 3. Creating the daily barangay-level dataset

**DO NOT SKIP THIS CODE BLOCK.** The output from this block is required to generate city-weekly tables.

This section creates a daily dataset at the barangay level, which will be aggregated to generate city-level tables.

> Disease data at a barangay-level is available in table `disease_lgu_disaggregated_totals` but only for selected years. You can explore the table offline by downloading it [here](https://drive.google.com/drive/u/1/folders/1NF_e3YM0dVBSxlVBWRxGQXwkl59sp5Jf)

Since some tables have temporal resolutions coarser than daily, the following adjustments were made:


- **Static (S)**: The single value is used to fill all included dates.

- **Yearly (Y)**: Yearly values for each barangay are used to fill years with available data, while the rest of the years are shown as NaN.

- **Monthly (M)**: Monthly values for each barangay are used to fill the months with available data, while the rest of the months are shown as NaN.

- **Daily (D)**: Values for each barangay are retained as is, and days without data are be shown as NaN.




In [ ]:
# @title 3.1 Linking barangay-level tables
%%time
assert 'columns' in locals(), "Parameters not yet selected. Please run previous cell."

# additions to tables and columns
base_tables_list = ['base_brgy']
combined_tables = base_tables_list + tables
core_columns_list = ['adm3_en', 'adm4_en', 'adm4_pcode', 'brgy_total_area', 'date']
combined_columns = core_columns_list + columns

# initialize lists
relevant_dataframes = []
static_columns = []
yearly_columns = []
monthly_columns = []

cities = [cities] if isinstance(cities, str) else cities
base_brgy = base_brgy[base_brgy['adm3_en'].isin(cities)]
cities_adm4_pcodes = base_brgy[base_brgy['adm3_en'].isin(cities)]['adm4_pcode'].unique()

# iterate through dataframe name and dataframe pairs to figure out frequency of columns
processed_dataframes = {}
for df_name, df in dfs_with_adm4_pcode:
    if df_name not in combined_tables:
      continue
    #df_copy = df.copy()
    df = df[df['adm4_pcode'].isin(cities_adm4_pcodes)]

    if df_name in combined_tables:
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')

        if df_name != 'base_brgy' and df_name != 'location' and not df_name.startswith('climate_'):
            base_brgy['date'] = pd.to_datetime(base_brgy['date'], format='%Y-%m-%d', errors='coerce')
            df = pd.merge(base_brgy[['adm4_pcode', 'date']], df, on=['adm4_pcode','date'], how='left')
            #filter for only barangays in cities

    relevant_dataframes.append((df_name, df))

    if df_name == 'base_brgy':
        df = df[df['adm3_en'].isin(cities)]

    if df_name in combined_tables:
        if 'freq' in df.columns:
            exclude_columns = ['adm3_en', 'adm3_pcode', 'adm3_en', 'adm4_pcode', 'date']
            unique_freq_values = df['freq'].dropna().unique()

            for freq in unique_freq_values:
                if pd.notna(freq):
                    freq_df = df[df['freq'] == freq]
                    if freq == 'S':
                        static_columns.extend([col for col in freq_df.columns if col not in exclude_columns and freq_df[col].nunique() > 1])
                    elif freq == 'Y':
                        yearly_columns.extend([col for col in freq_df.columns if col not in exclude_columns and freq_df[col].nunique() > 1])
                    elif freq == 'M':
                        monthly_columns.extend([col for col in freq_df.columns if col not in exclude_columns and freq_df[col].nunique() > 1])

# remove duplicates from the lists
static_columns = list(set(static_columns))
yearly_columns = list(set(yearly_columns))
monthly_columns = list(set(monthly_columns))

# pre-process dataframes before iterating for filling
for df_name, df in relevant_dataframes:
    if df_name in combined_tables:
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
        df['year'] = df['date'].dt.year
        df['year_month'] = df['date'].dt.to_period('M')

grouped_dfs = [(df_name, df.groupby(['date', 'year', 'year_month', 'adm4_pcode']))
               for df_name, df in relevant_dataframes if df_name in combined_tables]

# iterate through grouped dataframes to fill in missing values
for df_name, grouped_df in grouped_dfs:
    df_copy = grouped_df.first().reset_index().copy()

    # Static data
    for col in static_columns:
        if col in df_copy.columns:
            df_copy[col] = df_copy.groupby('adm4_pcode')[col].transform('first')

    # Yearly data
    for col in yearly_columns:
        if col in df_copy.columns:
            df_copy[col] = df_copy.groupby(['year', 'adm4_pcode'])[col].transform('first')

    # Monthly data
    for col in monthly_columns:
        if col in df_copy.columns:
            df_copy[col] = df_copy.groupby(['year_month', 'adm4_pcode'])[col].transform('first')

    df_copy = df_copy[(df_copy['date'] >= start_date) & (df_copy['date'] <= end_date)]
    columns_to_keep = [col for col in df_copy.columns if col in combined_columns]
    df_copy = df_copy[columns_to_keep]
    processed_dataframes[df_name] = df_copy
    globals()[df_name] = df_copy

# create new tuple only with selected tables
processed_dataframes_tuples = [(df_name, df) for df_name, df in processed_dataframes.items() if df_name in combined_tables]

# initialize merged_df with the first dataframe (base_brgy_df)
merged_df_name = 'base_brgy'
merged_df = globals()[merged_df_name]

# merge dataframes
for df_name, df in processed_dataframes_tuples[1:]:
    excluded_columns = ['freq', 'year', 'year_month', 'brgy_total_area']
    columns_to_merge = [col for col in df.columns if col not in excluded_columns]
    merged_df = pd.merge(merged_df, df[columns_to_merge], on=['adm4_pcode', 'date'], how='left')

# delete vars to free up memory
del processed_dataframes_tuples, grouped_dfs, processed_dataframes
# filter to selected cities
#merged_df = merged_df[merged_df['adm3_en'].isin(cities)]

barangay_merged_df = merged_df.sort_values('date')
barangay_merged_df

CPU times: user 2.06 s, sys: 64.6 ms, total: 2.13 s
Wall time: 2.15 s


,date,adm4_pcode,adm3_en,adm4_en,brgy_total_area,heat_index,pr,rh,solar_rad,tave,...,avg_rad_mean,clinic_nearest,dentist_nearest,doctors_nearest,hospital_nearest,optician_nearest,pharmacy_nearest,poi_count,pop_count_mean,pop_count_total
0,2014-01-01,PH137503001,City of Navotas,Sipac-Almacen,0.2223,27.11,0.0,71.85,207.09,26.01,...,15.301064,10000.0,10000.0,10000.0,10000.000000,10000.000000,0.000000,2.0,348.812033,8022.676758
13,2014-01-01,PH137503014,City of Navotas,Tanza,5.7566,27.11,0.0,71.85,207.09,26.01,...,2.196291,10000.0,10000.0,10000.0,10000.000000,10000.000000,1701.255440,3.0,105.887026,33672.074219
12,2014-01-01,PH137503013,City of Navotas,Tangos,0.3824,27.11,0.0,71.85,207.09,26.01,...,7.048459,10000.0,10000.0,10000.0,10000.000000,10000.000000,1894.267443,2.0,743.036237,34922.703125
11,2014-01-01,PH137503012,City of Navotas,San Roque,0.3004,27.11,0.0,71.85,207.09,26.01,...,12.322443,10000.0,10000.0,10000.0,10000.000000,10000.000000,1474.721160,3.0,426.140704,15767.206055
10,2014-01-01,PH137503011,City of Navotas,San Rafael Village,0.4287,27.11,0.0,71.85,207.09,26.01,...,17.849720,10000.0,10000.0,10000.0,10000.000000,9976.219236,2343.692431,2.0,730.534180,35065.640625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46005,2022-12-31,PH137503002,City of Navotas,Bagumbayan North,0.0676,28.24,0.0,76.22,145.35,26.47,...,27.900757,NaN,10000.0,10000.0,870.584756,10000.000000,764.031107,0.0,NaN,NaN
46004,2022-12-31,PH137503001,City of Navotas,Sipac-Almacen,0.2223,28.24,0.0,76.22,145.35,26.47,...,18.289280,NaN,10000.0,10000.0,110.135115,10000.000000,356.160356,14.0,NaN,NaN
46016,2022-12-31,PH137503013,City of Navotas,Tangos,0.3824,28.24,0.0,76.22,145.35,26.47,...,8.293758,NaN,10000.0,10000.0,1594.314742,10000.000000,735.553906,29.0,NaN,NaN
46009,2022-12-31,PH137503006,City of Navotas,Navotas East,0.0662,28.24,0.0,76.22,145.35,26.47,...,NaN,NaN,10000.0,10000.0,589.634092,10000.000000,838.644378,5.0,NaN,NaN


In [ ]:
# @title 3.2 Save as CSV in output folder
# @markdown Optional, run if you want to save the daily baranggay-level table (without health data) in a file

# save as csv
file_path = os.path.join(output_folder, "barangay_merged.csv")
barangay_merged_df.to_csv(file_path, index=False)

print(f"Barangay DataFrame has been saved as CSV in output folder.")
print("\n")
barangay_merged_df.info()

# check if city_or_barangay is equal to 'Barangay'
if city_or_barangay == 'Barangay':
    print("The notebook stops execution here since the option selected is `Barangay`.")
    print("Please select `City` and run all cells from the selection parameters again if you want city aggregates.")
    print("\n")
    raise SystemExit  # stop notebook execution if barangay-level only


Barangay DataFrame has been saved as CSV in output folder.


<class 'pandas.core.frame.DataFrame'>
Index: 46018 entries, 0 to 46017
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              46018 non-null  datetime64[ns]
 1   adm4_pcode        46018 non-null  object        
 2   adm3_en           46018 non-null  object        
 3   adm4_en           46018 non-null  object        
 4   brgy_total_area   46018 non-null  float64       
 5   heat_index        46018 non-null  float64       
 6   pr                46018 non-null  float64       
 7   rh                46018 non-null  float64       
 8   solar_rad         46018 non-null  float64       
 9   tave              46018 non-null  float64       
 10  tmax              46018 non-null  float64       
 11  tmin              46018 non-null  float64       
 12  uv_rad            46018 non-null  float64       
 13  wind_speed        46

# 4. Creating the weekly city-level table

The output of this section is a table aggregating daily barangay-level into weekly city-level datasets.

#### *Spatial aggregation*

In the spatial dimension, barangay-level data were aggregated to city-level in one of the three ways:

- **Sum**: This is applied to columns with `_sum` and `_count`. This represents a total count of the variable in the city (e.g. POIs)

- **Simple average**: This is applied to columns with `pct_`. This represents a mean of the parameter in the city (e.g. hazards, land cover),

- **Weighted average**: Applied to all other columns. This represents a mean weighted by the area covered by each baranggay.

### *Temporal aggregation*

In the temporal dimension, a **simple average** of daily data was obtained to come up with weekly aggregates.

<sub> *Note:* The code blocks in the succeeding sections may run for quite some time due to the processing requirements of aggregation functions.



In [ ]:
# @title 4.1. Aggregation

%%time

####### SPATIAL AGGREGATION #######

# define regular expressions for column names
sum_text_match = re.compile(r'(_sum|_count)') # maybe add total here
simple_avg_text_match = re.compile(r'(pct)')

columns_to_exclude = {'adm3_en', 'adm3_pcode', 'adm4_en', 'adm4_pcode', 'brgy_total_area', 'date'}
columns_to_exclude.update({col for col in columns_to_exclude if re.search(r'_pop_reached', col)})

# initialize lists
sum_columns = []
simple_avg_columns = []
weighted_avg_columns = []

# Iterate through DataFrames and their columns
for df_name, df in dfs_with_adm4_pcode:
    for col in df.columns:
        if col not in columns_to_exclude:
            if sum_text_match.search(col):
                sum_columns.append(col)
            elif simple_avg_text_match.search(col):
                simple_avg_columns.append(col)
            else:
                weighted_avg_columns.append(col)

# retain only the columns common with barangay_merged_df
sum_columns = [col for col in sum_columns if col in barangay_merged_df.columns]
simple_avg_columns = [col for col in simple_avg_columns if col in barangay_merged_df.columns]
weighted_avg_columns = [col for col in weighted_avg_columns if col in barangay_merged_df.columns]


print("\033[1mMETHODS OF AGGREGATION: \033[0m")
print("Sum:", sum_columns)
print("Simple Average:", simple_avg_columns)
print("Weighted Average:", weighted_avg_columns)
print("\n")


# sum functions
def sum_with_nan(values):
    valid_values = [value for value in values if not np.isnan(value)]
    return sum(valid_values) if valid_values else np.nan

def sum_function(df, columns):
    result = df.groupby(['adm3_en', 'date'])[columns].agg(sum_with_nan).reset_index()
    return result

# simple average function
def mean_with_nan(values):
    valid_values = [value for value in values if not np.isnan(value)]
    return sum(valid_values) / len(valid_values) if valid_values else np.nan

def simple_average_function(df, columns):
    result = df.groupby(['adm3_en', 'date'])[columns].agg(mean_with_nan).reset_index()
    return result

# weighted average function
def weighted_average_function(df, columns, area_column):
    def weighted_avg(col):
        col_without_nan = col.dropna()
        if len(col_without_nan) == 0: # Check if all values were NaN
            return np.nan
        else: # If not all NaN, calculate weighted average
            return (col_without_nan * df[area_column][col_without_nan.index]).sum() / df[area_column][col_without_nan.index].sum()

    # Apply the function
    result = df.groupby(['adm3_en', 'date'], as_index=False).apply(lambda x: pd.Series({
        col: weighted_avg(x[col]) # Calculate weighted average
        for col in columns
    })).reset_index(drop=True)

    return result

# create dataframes
sum_result = barangay_merged_df[['adm3_en', 'date'] + sum_columns]
simple_avg_result = barangay_merged_df[['adm3_en', 'date'] + simple_avg_columns]
weighted_avg_result = barangay_merged_df[['adm3_en', 'date', 'brgy_total_area'] + weighted_avg_columns]

# rename 'brgy_total_area' to 'city_total_area' if present in sum_result
if 'brgy_total_area' in sum_result.columns:
    sum_result.rename(columns={'brgy_total_area': 'city_total_area'}, inplace=True)

# update sum_columns
sum_columns = [col for col in sum_result.columns if col not in ['adm3_en', 'date']]

# apply functions
sum_result = sum_function(sum_result, sum_columns)
simple_avg_result = simple_average_function(simple_avg_result, simple_avg_columns)
weighted_avg_result = weighted_average_function(weighted_avg_result, weighted_avg_columns, 'brgy_total_area')

# convert date columns to datetime format
base_city['date'] = pd.to_datetime(base_city['date'])
sum_result['date'] = pd.to_datetime(sum_result['date'])
simple_avg_result['date'] = pd.to_datetime(simple_avg_result['date'])
weighted_avg_result['date'] = pd.to_datetime(weighted_avg_result['date'])

# filter base_city
base_city_filtered = base_city[base_city['adm3_en'].isin(cities)]

# merge dataframes
dfs_to_merge = [base_city_filtered, sum_result, simple_avg_result, weighted_avg_result]
barangay_processed_df = base_city_filtered
for df in dfs_to_merge[1:]:
    barangay_processed_df = pd.merge(barangay_processed_df, df, on=['adm3_en', 'date'], how='left')

# remove column if all values are NaN
barangay_processed_df = barangay_processed_df.dropna(axis=1, how='all')

####### TEMPORAL AGGREGATION #######

barangay_processed_week = barangay_processed_df.copy()
barangay_processed_week['date'] = pd.to_datetime(barangay_processed_week['date'])

# create 'year' and 'week' columns
barangay_processed_week['year'] = barangay_processed_week['date'].dt.year

def get_week_number(date):
    if date.strftime('%W') == '00':
        return (date - pd.DateOffset(days=7)).strftime('%W')
    else:
        return date.strftime('%W')

def get_year(date):
    if date.strftime('%W') == '00':
        year = (date - pd.DateOffset(days=7)).year
    else:
        year = date.year
    return year

barangay_processed_week['year'] = barangay_processed_week['date'].apply(get_year)
barangay_processed_week['week'] = barangay_processed_week['date'].apply(get_week_number)

# insert 'year' and 'week' columns after 'adm3_en'
adm3_en_index = barangay_processed_week.columns.get_loc('adm3_en')
barangay_processed_week.insert(adm3_en_index + 1, 'year', barangay_processed_week.pop('year'))
barangay_processed_week.insert(adm3_en_index + 2, 'week', barangay_processed_week.pop('week'))

# custom function to retain first date and mean for other columns
def custom_aggfunc(x):
    # if x.name == 'date' or x.dtype == 'O':
    if x.name == 'date' or isinstance(x.iloc[0], str):
        return x.iloc[0]
    else:
        return x.mean()

# pivot table
weekly_average_df = pd.pivot_table(barangay_processed_week,
                                   index=['adm3_en', 'adm3_pcode', 'year', 'week'],
                                   aggfunc=custom_aggfunc)

# reset index
weekly_average_df.reset_index(inplace=True)

####### ATTACH CITY LEVEL TABLES #######

# filter base_city_table
filtered_base_city = base_city[
    (base_city['date'] >= start_date) &
    (base_city['date'] <= end_date) &
    (base_city['adm3_en'].isin(cities))
]

# process dfs_without_adm4_pcode
result_dataframes = {}

for df_name, df in dfs_without_adm4_pcode:
    if df_name in city_tables:
        # Select columns based on city_columns and add disease columns if df is disease_weekly_totals
        selected_columns = ['adm3_pcode', 'date'] + [col for col in city_columns if col in df.columns]
        if df_name == 'disease_weekly_totals':
            selected_columns.extend(['disease_standard_name', 'disease_standard_code', 'source_name'])

        # Filter and format the dataframe
        filtered_df = df[selected_columns].copy()
        filtered_df['date'] = pd.to_datetime(filtered_df['date'], format='%Y-%m-%d')

        # Merge filtered dataframe with base city dataframe
        merged_df = pd.merge(filtered_df, filtered_base_city, on=['adm3_pcode', 'date'], how='inner')

        # If the dataframe is disease_weekly_totals, filter rows based on diseases
        if df_name == 'disease_weekly_totals':
            merged_df = merged_df[merged_df['disease_standard_name'].isin(diseases)]

        result_dataframes[df_name] = merged_df

# disease_weekly_totals_result = result_dataframes.get('disease_weekly_totals')
# disease_weekly_totals

# process health data
if 'disease_weekly_totals' in result_dataframes:
    updated_disease_weekly_totals = result_dataframes['disease_weekly_totals']

    updated_disease_weekly_totals['date'] = pd.to_datetime(updated_disease_weekly_totals['date'], format='%Y-%m-%d')
    updated_disease_weekly_totals = updated_disease_weekly_totals[~updated_disease_weekly_totals['source_name'].str.contains(r'FHSIS', regex=True)].drop(columns=['disease_standard_name', 'source_name'])

    # custom function for summation
    def sum_with_nan(values):
        valid_values = [value for value in values if isinstance(value, (int, float)) and not np.isnan(value)]
        return sum(valid_values) if valid_values else np.nan

    health_pivot_df = pd.pivot_table(updated_disease_weekly_totals,
                                      index=['date', 'adm3_pcode'],
                                      columns=['disease_standard_code'],
                                      aggfunc=sum_with_nan,
                                      fill_value=np.nan)

    health_pivot_df.columns = [f'{col[0]}_{col[1]}' if isinstance(col, tuple) else col for col in health_pivot_df.columns]
    health_pivot_df.reset_index(inplace=True)

    disease_mapping = {
        'A00': 'cholera',
        'A01.0': 'typhoid',
        'A08.0': 'rotavirus',
        'A09': 'gastroenteritis',
        'A09.0': 'abd',
        'A27': 'leptospirosis',
        'A82': 'rabies',
        'A92.0': 'chikungunya',
        'A97': 'dengue',
        'B15': 'hepatitis_a',
        'E08-E14': 'diabetes',
        'I11': 'hypertensive_cvd',
        'I20-I25': 'angina_pectoris',
        'I21-I22': 'ischemic_heart_diseases',
        'J45': 'asthma',
        'T67': 'heat_stroke'
    }

    new_column_names = []
    for col in health_pivot_df.columns:
        code = col.split('_')[-1]
        disease_name = disease_mapping.get(code, col)
        new_col = col.replace(code, disease_name)
        new_column_names.append(new_col)

    health_pivot_df.columns = new_column_names
    health_pivot_df.rename(columns={'adm3_adm3_pcode': 'adm3_pcode'}, inplace=True)

    result_dataframes['health_pivot_df'] = health_pivot_df

# initialize merged_all_df as weekly_average_df
merged_all_df = weekly_average_df.copy()

# check if both health_pivot_df and mapbox_health_facility_city_isochrones_df are present
if 'health_pivot_df' in result_dataframes and 'mapbox_health_facility_city_isochrones_df' in result_dataframes:

    # merging with health_pivot_df
    merged_df = pd.merge(weekly_average_df, result_dataframes['health_pivot_df'],
                         on=['adm3_pcode', 'date'], how='left')

    # dropping unnecessary columns from mapbox_health_facility_city_isochrones_df
    mapbox_health_facility_city_isochrones_df = result_dataframes['mapbox_health_facility_city_isochrones_df'].copy()
    mapbox_health_facility_city_isochrones_df.drop(columns=['date', 'adm3_en'], inplace=True, errors='ignore')

    # merging withmapbox_health_facility_city_isochrones_df
    merged_all_df = pd.merge(merged_df, mapbox_health_facility_city_isochrones_df,
                             on='adm3_pcode', how='left')

# if only health_pivot_df is present
elif 'health_pivot_df' in result_dataframes:
    merged_all_df = pd.merge(weekly_average_df, result_dataframes['health_pivot_df'],
                             on=['adm3_pcode', 'date'], how='left')

# if only mapbox_health_facility_city_isochrones is present
elif 'mapbox_health_facility_city_isochrones' in result_dataframes:
    mapbox_health_facility_city_isochrones = result_dataframes['mapbox_health_facility_city_isochrones'].copy()
    mapbox_health_facility_city_isochrones.drop(columns=['date', 'adm3_en'], inplace=True, errors='ignore')
    merged_all_df = pd.merge(weekly_average_df, mapbox_health_facility_city_isochrones,
                             on='adm3_pcode', how='left')

# filter to show only values within specified dates
merged_all_filtered_df = merged_all_df[(merged_all_df['date'] >= start_date) & (merged_all_df['date'] <= end_date)]

# rearrange columns
desired_column_order = ['adm3_en', 'adm3_pcode', 'date', 'year', 'week'] + [col for col in merged_all_filtered_df.columns if col not in ['adm3_en', 'adm3_pcode', 'date', 'year', 'week']]
merged_all_filtered_df = merged_all_filtered_df[desired_column_order]

merged_all_filtered_df

METHODS OF AGGREGATION: 
Sum: ['poi_count', 'pop_count_mean', 'pop_count_total']
Simple Average: []
Weighted Average: ['heat_index', 'pr', 'rh', 'solar_rad', 'tave', 'tmax', 'tmin', 'uv_rad', 'wind_speed', 'avg_rad_mean', 'clinic_nearest', 'dentist_nearest', 'doctors_nearest', 'hospital_nearest', 'optician_nearest', 'pharmacy_nearest']


CPU times: user 1min 8s, sys: 2.77 s, total: 1min 11s
Wall time: 1min 9s


,adm3_en,adm3_pcode,date,year,week,avg_rad_mean,clinic_nearest,dentist_nearest,doctors_nearest,heat_index,...,pop_count_mean,pop_count_total,pr,rh,solar_rad,tave,tmax,tmin,uv_rad,wind_speed
575,City of Navotas,PH137503000,2014-01-06,2014,01,9.196975,NaN,10000.000000,10000.000000,26.744286,...,5837.02747,279489.947754,0.188900,70.081429,192.047143,25.800000,28.271429,23.894286,22.904286,3.758571
576,City of Navotas,PH137503000,2014-01-13,2014,02,9.196975,NaN,10000.000000,10000.000000,24.744286,...,5837.02747,279489.947754,0.705377,66.171429,180.272857,24.381429,26.687143,22.805714,21.132857,4.014286
577,City of Navotas,PH137503000,2014-01-20,2014,03,9.196975,NaN,10000.000000,10000.000000,23.748571,...,5837.02747,279489.947754,0.000000,63.317143,200.181429,23.635714,26.428571,21.410000,23.150000,3.328571
578,City of Navotas,PH137503000,2014-01-27,2014,04,9.196975,NaN,10000.000000,10000.000000,25.640000,...,5837.02747,279489.947754,1.045361,68.617143,191.987143,24.988571,28.090000,22.667143,22.448571,2.475714
579,City of Navotas,PH137503000,2014-02-03,2014,05,9.196975,NaN,10000.000000,10000.000000,29.250000,...,5837.02747,279489.947754,0.000000,73.471429,201.622857,27.410000,30.787143,24.538571,24.141429,1.398571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,City of Navotas,PH137503000,2022-11-28,2022,48,12.920640,NaN,9722.709984,9873.015621,32.941429,...,NaN,NaN,1.537057,74.445714,183.807143,28.861429,31.817143,26.595714,21.994286,2.168571
1040,City of Navotas,PH137503000,2022-12-05,2022,49,12.920640,NaN,9722.709984,9873.015621,31.067143,...,NaN,NaN,4.939297,78.534286,150.428571,27.812857,30.228571,25.917143,18.344286,1.132857
1041,City of Navotas,PH137503000,2022-12-12,2022,50,12.920640,NaN,9722.709984,9873.015621,31.140000,...,NaN,NaN,1.653142,77.372857,171.312857,27.854286,30.171429,26.011429,20.954286,2.012857
1042,City of Navotas,PH137503000,2022-12-19,2022,51,12.920640,NaN,9722.709984,9873.015621,26.858571,...,NaN,NaN,1.093462,73.148571,107.905714,25.751429,27.755714,24.415714,13.027143,2.744286


In [ ]:
# @title 4.2. Save as CSV in output folder

# save as csv
file_path = os.path.join(output_folder, "city_merged.csv")
merged_all_filtered_df.to_csv(file_path, index=False)

print(f"City DataFrame has been saved as CSV in output folder.")
print("\n")

merged_all_filtered_df.info()

City DataFrame has been saved as CSV in output folder.


<class 'pandas.core.frame.DataFrame'>
Index: 469 entries, 575 to 1043
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   adm3_en           469 non-null    object        
 1   adm3_pcode        469 non-null    object        
 2   date              469 non-null    datetime64[ns]
 3   year              469 non-null    int64         
 4   week              469 non-null    object        
 5   avg_rad_mean      469 non-null    float64       
 6   clinic_nearest    8 non-null      float64       
 7   dentist_nearest   469 non-null    float64       
 8   doctors_nearest   469 non-null    float64       
 9   heat_index        469 non-null    float64       
 10  hospital_nearest  469 non-null    float64       
 11  optician_nearest  469 non-null    float64       
 12  pharmacy_nearest  469 non-null    float64       
 13  poi_count         469 non